<a href="https://colab.research.google.com/github/paddy0914/Colab/blob/main/Hybrid_Demucs_Music_Source_Separation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Demucs from Colab

This supports the Demucs source separation model (https://github.com/facebookresearch/demucs/)
This is only for separation with pre-trained models, not training!

You can either upload files manually (slow) or link your Google Drive account.

In [1]:
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-iyly5xec/demucs_b384cb49e6694bb29d81921145aaaace
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-iyly5xec/demucs_b384cb49e6694bb29d81921145aaaace
  Resolved https://github.com/facebookresearch/demucs to commit 404690cb2555d767312cb2cee8a68a6d7bff018d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 1.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 6.9 MB/s eta

In [2]:
# Please BE VERY CAREFUL, this will link your entire drive.
# So don't edit code, except the one that says 'Customize the following options',
# or you might mess up your files.
# IF YOU DO NO WANT TO LINK DRIVE, please see below for an alternative!
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
# Customize the following options!
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 48000
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

in_path = '/gdrive/MyDrive/dataset_audio/raw_audio/'
out_path = '/gdrive/MyDrive/dataset_audio/denoised_audio/'

In [6]:
#@title Useful functions, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

from google.colab import files

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")


def from_upload():
    out_path = Path('separated')
    in_path = Path('tmp_in')

    if in_path.exists():
        rmtree(in_path)
    in_path.mkdir()

    if out_path.exists():
        rmtree(out_path)
    out_path.mkdir()

    uploaded = files.upload()
    for name, content in uploaded.items():
        (in_path / name).write_bytes(content)
    separate(in_path, out_path)


In [7]:
# This can be quite slow, in particular the loading, and saving from GDrive. Please be patient!
# This is from google drive! Also, this will separate all the files inside the MyDrive/demucs folder,
# so when you are happy with the results, remove the songs from there.
separate()

Going to separate the files:
/gdrive/MyDrive/dataset_audio/raw_audio/PMHDSSMAINLANDQA(IV)846_6600C5A8-E363-451B-8899-80906A864859.mp3
/gdrive/MyDrive/dataset_audio/raw_audio/PMHDSSMAINLANDQA(IV)861_24A1AB1E-3ABE-4BE9-B149-004A708E4A22.mp3
With command:  python3 -m demucs.separate -o /gdrive/MyDrive/dataset_audio/denoised_audio/ -n htdemucs --mp3 --mp3-bitrate=48000


Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
100%|██████████| 80.2M/80.2M [00:00<00:00, 141MB/s]


Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /gdrive/MyDrive/dataset_audio/denoised_audio/htdemucs
Separating track /gdrive/MyDrive/dataset_audio/raw_audio/PMHDSSMAINLANDQA(IV)846_6600C5A8-E363-451B-8899-80906A864859.mp3


100%|██████████████████████████████████████████████████████████████████████| 1567.8/1567.8 [01:06<00:00, 23.61seconds/s]


Separating track /gdrive/MyDrive/dataset_audio/raw_audio/PMHDSSMAINLANDQA(IV)861_24A1AB1E-3ABE-4BE9-B149-004A708E4A22.mp3


100%|██████████████████████████████████████████████| 1544.3999999999999/1544.3999999999999 [01:01<00:00, 25.10seconds/s]


In [ ]:
# This is manual upload and download :)
from_upload()
!zip -r separated.zip separated
files.download('./separated.zip')